# General data exploration and cleaning

In [1]:
import pandas as pd
import numpy as np

# Dataset
df = pd.read_csv("flights.csv")

# Initial exploration
print("Dataset shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nDate range:", df['YEAR'].min(), "to", df['YEAR'].max())
print("\nCountries:", df['STATE_NAME'].nunique())
print("\nData Types:")
print(df.dtypes)

Dataset shape: (688099, 14)

Columns: ['YEAR', 'MONTH_NUM', 'MONTH_MON', 'FLT_DATE', 'APT_ICAO', 'APT_NAME', 'STATE_NAME', 'FLT_DEP_1', 'FLT_ARR_1', 'FLT_TOT_1', 'FLT_DEP_IFR_2', 'FLT_ARR_IFR_2', 'FLT_TOT_IFR_2', 'Pivot Label']

Date range: 2016 to 2022

Countries: 42

Data Types:
YEAR               int64
MONTH_NUM          int64
MONTH_MON         object
FLT_DATE          object
APT_ICAO          object
APT_NAME          object
STATE_NAME        object
FLT_DEP_1          int64
FLT_ARR_1          int64
FLT_TOT_1          int64
FLT_DEP_IFR_2    float64
FLT_ARR_IFR_2    float64
FLT_TOT_IFR_2    float64
Pivot Label       object
dtype: object


In [2]:
print("\nSample data:")
print(df.head())


Sample data:
   YEAR  MONTH_NUM MONTH_MON              FLT_DATE APT_ICAO       APT_NAME  \
0  2016          1       JAN  2016-01-01T00:00:00Z     EBAW        Antwerp   
1  2016          1       JAN  2016-01-01T00:00:00Z     EBBR       Brussels   
2  2016          1       JAN  2016-01-01T00:00:00Z     EBCI      Charleroi   
3  2016          1       JAN  2016-01-01T00:00:00Z     EBLG          Liège   
4  2016          1       JAN  2016-01-01T00:00:00Z     EBOS  Ostend-Bruges   

  STATE_NAME  FLT_DEP_1  FLT_ARR_1  FLT_TOT_1  FLT_DEP_IFR_2  FLT_ARR_IFR_2  \
0    Belgium          4          3          7            NaN            NaN   
1    Belgium        174        171        345          174.0          161.0   
2    Belgium         45         47         92           45.0           45.0   
3    Belgium          6          7         13            NaN            NaN   
4    Belgium          7          7         14            NaN            NaN   

   FLT_TOT_IFR_2           Pivot Label  
0

In [3]:
# Convert FLT_DATE from string to datetime
df['FLT_DATE'] = pd.to_datetime(df['FLT_DATE'])

In [4]:
# Verify
print(df.dtypes)

YEAR                           int64
MONTH_NUM                      int64
MONTH_MON                     object
FLT_DATE         datetime64[ns, UTC]
APT_ICAO                      object
APT_NAME                      object
STATE_NAME                    object
FLT_DEP_1                      int64
FLT_ARR_1                      int64
FLT_TOT_1                      int64
FLT_DEP_IFR_2                float64
FLT_ARR_IFR_2                float64
FLT_TOT_IFR_2                float64
Pivot Label                   object
dtype: object


In [5]:
# Missing values
print(df.isnull().sum())

YEAR                  0
MONTH_NUM             0
MONTH_MON             0
FLT_DATE              0
APT_ICAO              0
APT_NAME              0
STATE_NAME            0
FLT_DEP_1             0
FLT_ARR_1             0
FLT_TOT_1             0
FLT_DEP_IFR_2    479785
FLT_ARR_IFR_2    479785
FLT_TOT_IFR_2    479785
Pivot Label           0
dtype: int64


Problematic fields:
- FLT_DEP_IFR_2    479785
- FLT_ARR_IFR_2    479785
- FLT_TOT_IFR_2    479785

According to dataset, these three fields should report the same information as flt_dep_1, flt_arr_1, flt_tot_1

Where rows have values in both sets, compare them to see if the former set can be dropped.

In [6]:
# Compare the two

# Check number of rows
has_both_sources = df[['FLT_DEP_1', 'FLT_DEP_IFR_2']].notna().all(axis=1)
print(f"Rows with both data sources: {has_both_sources.sum()} ({has_both_sources.mean()*100:.1f}%)")

Rows with both data sources: 208314 (30.3%)


In [7]:
# Compare values where both exist
if has_both_sources.any():
    comparison = df[has_both_sources].copy()
    comparison['DEP_DIFF'] = comparison['FLT_DEP_1'] - comparison['FLT_DEP_IFR_2']
    comparison['ARR_DIFF'] = comparison['FLT_ARR_1'] - comparison['FLT_ARR_IFR_2']
    comparison['TOT_DIFF'] = comparison['FLT_TOT_1'] - comparison['FLT_TOT_IFR_2']
    
    print("\nDifference statistics (Source1 - Source2):")
    print(f"Departures - Mean diff: {comparison['DEP_DIFF'].mean():.2f}, Std: {comparison['DEP_DIFF'].std():.2f}")
    print(f"Arrivals - Mean diff: {comparison['ARR_DIFF'].mean():.2f}, Std: {comparison['ARR_DIFF'].std():.2f}")
    print(f"Total - Mean diff: {comparison['TOT_DIFF'].mean():.2f}, Std: {comparison['TOT_DIFF'].std():.2f}")
    
    # Check if differences are significant
    significant_diff = (abs(comparison['TOT_DIFF']) > 5).sum()
    print(f"Rows with significant difference (>5 flights): {significant_diff}")



Difference statistics (Source1 - Source2):
Departures - Mean diff: 0.52, Std: 7.21
Arrivals - Mean diff: 0.66, Std: 6.84
Total - Mean diff: 1.18, Std: 12.69
Rows with significant difference (>5 flights): 33357


The dataset contains two measurement sources (Network Manager vs Airport Operator). While mean differences were minimal (0.5-1.2 flights), significant discrepancies (>5 flights) occurred in 33,357 records (4.8% of data). This informed my decision to use Network Manager data (FLT_*_1) as the primary source due to its completeness, while acknowledging potential measurement variance.

Rename for clarity:

In [ ]:
df = df.rename(columns={
    # Primary flight data (what I will be using)
    'FLT_DEP_1': 'departures',
    'FLT_ARR_1': 'arrivals', 
    'FLT_TOT_1': 'total_flights',
    
    # Secondary flight data (what I will not be using)
    'FLT_DEP_IFR_2': 'departures_secondary',
    'FLT_ARR_IFR_2': 'arrivals_secondary', 
    'FLT_TOT_IFR_2': 'total_flights_secondary',
    
    # Geographic fields
    'STATE_NAME': 'country',
    'APT_NAME': 'airport',
    'APT_ICAO': 'airport_code',
    'Pivot Label': 'airport_name_and_code',
    
    # Time fields
    'MONTH_MON': 'month',
    'MONTH_NUM': 'month_num',
    'FLT_DATE': 'date',
    'YEAR': 'year'
})

print("New column names:", df.columns.tolist())

New column names: ['year', 'month_num', 'month', 'date', 'airport_code', 'airport', 'country', 'departures', 'arrivals', 'total_flights', 'departures_secondary', 'arrivals_secondary', 'total_flights_secondary', 'airport_name_and_code']


In [ ]:
duplicates = df.duplicated(subset=['date','airport_code'])
print(f"Duplicate rows: {duplicates.sum()}")

Duplicate rows: 0


Check for negative or zero flights

In [10]:
print((df[['departures','arrivals','total_flights']] < 0).sum())


departures       0
arrivals         0
total_flights    0
dtype: int64


Verify departures + arrivals = total_flights

In [11]:
diff = df['departures'] + df['arrivals'] - df['total_flights']
print(f"Rows with discrepancy: {(diff != 0).sum()}")


Rows with discrepancy: 0


In [12]:
# Detect non-ASCII characters
def has_non_ascii(s):
    return any(ord(c) > 127 for c in s)

# Filter rows with unusual characters
unusual_countries = df['country'][df['country'].apply(has_non_ascii)].unique()

print("Countries with unusual characters:", unusual_countries)

Countries with unusual characters: ['Türkiye']


In [13]:
# Normalise country names
df['country'] = df['country'].replace({'Türkiye': 'Turkey'})

# Chart specific data preparation

### CHART 1: National Growth Trends (Line Chart)

In [14]:
# Get yearly flight totals per country
chart1_data = df.groupby(['country', 'year']).agg({
    'total_flights': 'sum'
}).reset_index()

chart1_data = chart1_data.sort_values(['country', 'year'])

# Calculate year-over-year percentage change (growth) for each country
chart1_data['%_change'] = chart1_data.groupby('country')['total_flights'].pct_change() * 100

print("Chart 1 data preview:")
print(chart1_data.head(20))


Chart 1 data preview:
    country  year  total_flights    %_change
0   Albania  2016          22319         NaN
1   Albania  2017          24559   10.036292
2   Albania  2018          25515    3.892667
3   Albania  2019          28500   11.699001
4   Albania  2020          15288  -46.357895
5   Albania  2021          26190   71.310832
6   Albania  2022          12879  -50.824742
7   Armenia  2016          18406         NaN
8   Armenia  2017          21210   15.234163
9   Armenia  2018          23011    8.491278
10  Armenia  2019          25556   11.059928
11  Armenia  2020           9373  -63.323681
12  Armenia  2021          19439  107.393577
13  Armenia  2022           9795  -49.611606
14  Austria  2016         326052         NaN
15  Austria  2017         327886    0.562487
16  Austria  2018         343331    4.710479
17  Austria  2019         365761    6.533054
18  Austria  2020         151426  -58.599741
19  Austria  2021         172705   14.052408


Earliest year (2016) will always show NAN as there is no previous year (2015) in the dataset to compare it to. Let's remove 2016 data.

In [15]:
print(f"Before: {len(chart1_data)} rows")

Before: 290 rows


In [16]:
# Remove null percentage changes (2016 data)
chart1_data = chart1_data[chart1_data['%_change'].notna()].copy()

print(f"After removing 2016 (null %_change): {len(chart1_data)} rows")

After removing 2016 (null %_change): 248 rows


### For CHART 2: Seasonal Patterns (Vertical bar chart)

In [17]:
# Seasons
season_map = {
    1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring', 6: 'Summer',
    7: 'Summer', 8: 'Summer', 9: 'Autumn', 10: 'Autumn', 11: 'Autumn', 12: 'Winter'
}
df['season'] = df['month_num'].map(season_map)

# Order for plotting purposes
season_order = ['Winter','Spring','Summer','Autumn']
df['season'] = pd.Categorical(df['season'], categories=season_order, ordered=True)

# Get total flights per country per season for each individual year
country_season_year = df.groupby(['country', 'season', 'year']).agg({
    'total_flights': 'sum',   
    'departures': 'sum',   # for arrival/departure split
    'arrivals': 'sum'    
}).reset_index()

# Average above result across years to get typical seasonal trend for each country
chart2_data = country_season_year.groupby(['country', 'season']).agg({
    'total_flights': 'mean',   
    'departures': 'mean',
    'arrivals': 'mean'
}).reset_index()

print("Chart 2 data preview (average seasonal flights per year per country):")
print(chart2_data.head(20))

Chart 2 data preview (average seasonal flights per year per country):
                   country  season  total_flights    departures      arrivals
0                  Albania  Winter    5088.142857   2544.142857   2544.000000
1                  Albania  Spring    5252.000000   2624.285714   2627.714286
2                  Albania  Summer    6378.571429   3187.714286   3190.857143
3                  Albania  Autumn    5459.857143   2731.285714   2728.571429
4                  Armenia  Winter    4278.428571   2140.857143   2137.571429
5                  Armenia  Spring    4533.857143   2268.571429   2265.285714
6                  Armenia  Summer    4803.428571   2406.428571   2397.000000
7                  Armenia  Autumn    4497.142857   2252.285714   2244.857143
8                  Austria  Winter   60778.000000  30379.285714  30398.714286
9                  Austria  Spring   64921.285714  32439.714286  32481.571429
10                 Austria  Summer   66256.714286  33102.428571  33154.2

C:\Users\alicj\AppData\Local\Temp\ipykernel_30092\3372173180.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  country_season_year = df.groupby(['country', 'season', 'year']).agg({
C:\Users\alicj\AppData\Local\Temp\ipykernel_30092\3372173180.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chart2_data = country_season_year.groupby(['country', 'season']).agg({


#### Additionally, for default chart view we will show overall European seasonal flight traffic, only when user searches for a specific country, they will get seasonal trends for that country only (utilising the above code).

In [18]:
# DEFAULT VIEW FOR CHART 2: Seasonal trends for all of Europe

# Aggregate total flights per season per year (all countries combined)
europe_season_year = df.groupby(['season', 'year']).agg({
    'total_flights': 'sum',
    'departures': 'sum',
    'arrivals': 'sum'
}).reset_index()

# Average across years for typical seasonal profile for Europe
europe_seasonal_trends = europe_season_year.groupby('season').agg({
    'total_flights': 'mean',
    'departures': 'mean',
    'arrivals': 'mean'
}).reset_index()

print("\nEuropean Seasonal Trends (default for Chart 2):")
print(europe_seasonal_trends)



European Seasonal Trends (default for Chart 2):
   season  total_flights    departures      arrivals
0  Winter   2.824854e+06  1.412152e+06  1.412702e+06
1  Spring   3.170743e+06  1.584495e+06  1.586249e+06
2  Summer   3.341000e+06  1.670039e+06  1.670960e+06
3  Autumn   3.100051e+06  1.549687e+06  1.550364e+06


C:\Users\alicj\AppData\Local\Temp\ipykernel_30092\2030032.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  europe_season_year = df.groupby(['season', 'year']).agg({
C:\Users\alicj\AppData\Local\Temp\ipykernel_30092\2030032.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  europe_seasonal_trends = europe_season_year.groupby('season').agg({


In [19]:
# Add Europe seasonal trends into chart2_data
europe_seasonal_trends['country'] = 'Europe'

# Ensure column order matches chart2_data
europe_seasonal_trends = europe_seasonal_trends[['country', 'season', 'total_flights', 'departures', 'arrivals']]

chart2_data = pd.concat([chart2_data, europe_seasonal_trends], ignore_index=True)

### For CHART 3: Airport Analysis (Horizontal bar chart)

In [20]:
# Get total flights per airport per country for each individual year
airport_year = df.groupby(['country', 'airport', 'year']).agg({
    'total_flights': 'sum',
    'departures': 'sum',
    'arrivals': 'sum'
}).reset_index()

# Average above result across years to get typical year traffic for each airport
chart3_data = airport_year.groupby(['country', 'airport']).agg({
    'total_flights': 'mean',
    'departures': 'mean',
    'arrivals': 'mean'
}).reset_index()

print("Chart 3 data preview (average yearly traffic per airport per country):")
print(chart3_data.head(20))

Chart 3 data preview (average yearly traffic per airport per country):
                   country        airport  total_flights    departures  \
0                  Albania         Tirana   22178.571429  11087.428571   
1                  Armenia        Yerevan   18112.857143   9068.142857   
2                  Austria           Graz   14181.428571   7069.714286   
3                  Austria      Innsbruck   14594.857143   7290.714286   
4                  Austria     Klagenfurt    5278.571429   2643.571429   
5                  Austria           Linz    9722.857143   4830.714286   
6                  Austria       Salzburg   21829.857143  10895.142857   
7                  Austria         Vienna  189267.285714  94631.428571   
8                  Belgium        Antwerp   12215.714286   6101.714286   
9                  Belgium       Brussels  168356.428571  84174.142857   
10                 Belgium      Charleroi   41222.285714  20626.714286   
11                 Belgium          Liège

#### Additionally, for default chart view we will show which top airports drive traffic in Europe, only when user searches for a specific country, they will see which airports in that specific country drive its traffic (utilising the above code).

In [21]:
# DEFAULT VIEW FOR CHART 3: Top airports across Europe based on average yearly traffic

# First compute total flights per airport per year (all countries combined)
airport_europe_year = df.groupby(['airport', 'year']).agg({
    'total_flights': 'sum',
    'departures': 'sum',
    'arrivals': 'sum'
}).reset_index()

# Average across years for a typical year
airport_europe_avg = airport_europe_year.groupby(['airport']).agg({
    'total_flights': 'mean',
    'departures': 'mean',
    'arrivals': 'mean'
}).reset_index()

# Select top N airports
N = 20
top_europe_airports = airport_europe_avg.sort_values('total_flights', ascending=False).head(N)

print(f"\nTop {N} European Airports by yearly traffic (default for Chart 3):")
print(top_europe_airports)



Top 20 European Airports by yearly traffic (default for Chart 3):
                     airport  total_flights     departures       arrivals
12      Amsterdam - Schiphol  384860.714286  192461.142857  192399.571429
223  Paris-Charles-de-Gaulle  368810.428571  184467.714286  184342.714286
97                 Frankfurt  368789.142857  184378.571429  184410.571429
167        London - Heathrow  349496.285714  174729.714286  174766.571429
179         Madrid - Barajas  302437.571429  151264.571429  151173.000000
202                   Munich  287646.285714  143837.714286  143808.571429
24                 Barcelona  242823.714286  121425.000000  121398.714286
123         Istanbul Atatürk  224355.428571  112180.714286  112174.714286
247         Rome - Fiumicino  216871.428571  108442.000000  108429.428571
331                   Zürich  196045.857143   98041.571429   98004.285714
166         London - Gatwick  191442.428571   95716.857143   95725.571429
77      Copenhagen - Kastrup  190846.571429  

In [22]:
# Label the rows as 'Europe' for default view
top_europe_airports['country'] = 'Europe'

# Match column order of chart3_data
top_europe_airports = top_europe_airports[['country', 'airport', 'total_flights', 'departures', 'arrivals']]

# Append to chart3_data
chart3_data = pd.concat([chart3_data, top_europe_airports], ignore_index=True)


In [23]:
# Pre-calculate top 20 airports per country (max 20 top, some countries will have less airports), including Europe
chart3_data = chart3_data.groupby('country', group_keys=False).apply(
    lambda x: x.sort_values('total_flights', ascending=False).head(20)
).reset_index(drop=True)

C:\Users\alicj\AppData\Local\Temp\ipykernel_30092\1488229362.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  chart3_data = chart3_data.groupby('country', group_keys=False).apply(


#### Final validation

In [24]:
print(f"Chart 1 (Trends): {chart1_data['country'].nunique()} countries, {chart1_data['year'].nunique()} years")

print(f"Chart 2 (Seasonality): {chart2_data['country'].nunique()} countries, {chart2_data['season'].nunique()} seasons")

print(f"Chart 3 (Airport Analysis): {chart3_data['airport'].nunique()} airports")

Chart 1 (Trends): 42 countries, 6 years
Chart 2 (Seasonality): 43 countries, 4 seasons
Chart 3 (Airport Analysis): 232 airports


#### Export for vega lite

In [25]:
# Export separate files for each chart
chart1_data.to_csv('chart1_country_trends.csv', index=False)
chart2_data.to_csv('chart2_seasonal_patterns.csv', index=False) 
chart3_data.to_csv('chart3_airport_breakdown.csv', index=False)

In [ ]:
# Create single combined CSV for all charts
# Add chart identifiers to each dataset
chart1_data['chart_type'] = 'country_trends'
chart2_data['chart_type'] = 'seasonal_patterns' 
chart3_data['chart_type'] = 'airport_breakdown'

# Select only needed columns
final_data = pd.concat([
    chart1_data[['country', 'year', 'total_flights', '%_change', 'chart_type']],
    chart2_data[['country', 'season', 'total_flights', 'departures', 'arrivals', 'chart_type']],
    chart3_data[['country', 'airport', 'total_flights', 'departures', 'arrivals', 'chart_type']] 
], ignore_index=True)

# Make sure year is an integer
final_data['year'] = final_data['year'].fillna(0).astype(int)
print(final_data.dtypes)

# Export the single final csv
final_data.to_csv('final_flights.csv', index=False)

print("\nFinal single CSV:")
print(f"Final CSV size: {len(final_data)} rows")
print(f"Chart breakdown:")
print(f"Country trends: {len(final_data[final_data['chart_type'] == 'country_trends'])} rows")
print(f"Seasonal patterns: {len(final_data[final_data['chart_type'] == 'seasonal_patterns'])} rows")
print(f"Airport breakdown: {len(final_data[final_data['chart_type'] == 'airport_breakdown'])} rows")

country            object
year                int64
total_flights     float64
%_change          float64
chart_type         object
season           category
departures        float64
arrivals          float64
airport            object
dtype: object

Final single CSV:
Final CSV size: 672 rows
Chart breakdown:
Country trends: 248 rows
Seasonal patterns: 172 rows
Airport breakdown: 252 rows
